In [1]:
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR


2023-01-27 10:43:54,001 [11564] WARNING  py.warnings:109: [JupyterRequire] /Users/javi/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



# Saco el dataframe global con los links para poder predecir y despues hacer la playlist

In [2]:
datos_spotipy_week_2 = pd.read_csv('../Data/CSV_full/data_hackshow_global')

In [3]:
datos_spotipy_week_2

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,links
0,Sam Smith,I’m Not Here To Make Friends (feat. Calvin Har...,0.895,0.698,-4.439,0.17200,0.0628,0.4090,0.838,114.914,229.60,https://open.spotify.com/track/3i0FkJYlU4MFfYk...
1,ROSALÍA,LLYLM,0.634,0.555,-5.522,0.12900,0.2730,0.1880,0.555,170.228,174.04,https://open.spotify.com/track/2SiAcexM2p1yX6j...
2,The Kid LAROI,Love Again,0.398,0.662,-6.691,0.71600,0.0275,0.1110,0.468,107.001,145.85,https://open.spotify.com/track/4sx6NRwL6Ol3V6m...
3,Ava Max,Ghost,0.931,0.715,-3.548,0.00573,0.0426,0.0549,0.615,119.971,181.65,https://open.spotify.com/track/7LbhIjYNfCguxjA...
4,Cordae,Two Tens (feat. Anderson .Paak),0.546,0.603,-7.525,0.19400,0.3500,0.0783,0.667,87.990,152.37,https://open.spotify.com/track/6clDsO8HwhHEgJD...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Ray Vaughn,Sandcastles,0.701,0.802,-5.564,0.03410,0.2140,0.2050,0.476,135.045,170.66,https://open.spotify.com/track/4Pbygk2v96q46Gc...
96,ghost orchard,all in,0.498,0.716,-6.643,0.71900,0.1560,0.1070,0.521,84.252,83.13,https://open.spotify.com/track/58tzJQpMWgsPaWm...
97,Prentiss,i tried,0.568,0.399,-9.813,0.42800,0.1190,0.2440,0.265,119.978,220.04,https://open.spotify.com/track/1lCm0JDZgZ2gyoJ...
98,Red Leather,DAKOTA,0.723,0.626,-4.464,0.14800,0.0356,0.0695,0.592,173.016,166.22,https://open.spotify.com/track/5WXlPR87hReTmOB...


In [4]:
datos_spotipy_week_2norm = datos_spotipy_week_2.copy()

In [5]:
datos_spotipy_week_2norm['artist_name']=LabelEncoder().fit_transform(datos_spotipy_week_2norm['artist_name'])


In [6]:
scaler=StandardScaler()

In [7]:
datos_spotipy_week_2norm[['energy','danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration','artist_name']] = scaler.fit_transform(datos_spotipy_week_2norm[['energy',\
                    'danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration','artist_name']])

In [8]:
datos_spotipy_week_2norm.drop(['track_name','links'],axis=1,inplace=True)

In [9]:
datos_spotipy_week_2norm.head()

,artist_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,0.610074,1.107956,0.458115,0.740934,-0.123326,-0.390903,1.839740,1.511719,-0.242794,0.767498
1,0.218773,-0.220254,-0.564048,0.304764,-0.294342,1.922056,0.126580,0.204990,1.636687,-0.436634
2,0.930230,-1.421240,0.200787,-0.166043,2.040215,-0.779331,-0.470313,-0.196725,-0.511665,-1.047586
3,-1.631014,1.291157,0.579631,1.099778,-0.784599,-0.613176,-0.905192,0.482035,-0.070966,-0.271706
4,-1.132995,-0.668079,-0.220945,-0.501930,-0.035830,2.769334,-0.723798,0.722141,-1.157629,-0.906280


In [10]:
pickled_model = pickle.load(open('../Data/CSV_primeros/HGBT.pkl', 'rb'))


In [11]:
y_pred_week2 = pickled_model.predict(datos_spotipy_week_2norm)

In [12]:
pred_week2= pd.Series(y_pred_week2) 

In [13]:
datos_spotipy_week_2['prediccion'] = pred_week2.round(decimals = 0)

In [14]:
datos_spotipy_week_2.prediccion.value_counts()

0    56
1    44
Name: prediccion, dtype: int64

In [15]:
datos_spotipy_week_2[['artist_name','prediccion','track_name','links']]

,artist_name,prediccion,track_name,links
0,Sam Smith,0,I’m Not Here To Make Friends (feat. Calvin Har...,https://open.spotify.com/track/3i0FkJYlU4MFfYk...
1,ROSALÍA,1,LLYLM,https://open.spotify.com/track/2SiAcexM2p1yX6j...
2,The Kid LAROI,0,Love Again,https://open.spotify.com/track/4sx6NRwL6Ol3V6m...
3,Ava Max,1,Ghost,https://open.spotify.com/track/7LbhIjYNfCguxjA...
4,Cordae,0,Two Tens (feat. Anderson .Paak),https://open.spotify.com/track/6clDsO8HwhHEgJD...
...,...,...,...,...
95,Ray Vaughn,1,Sandcastles,https://open.spotify.com/track/4Pbygk2v96q46Gc...
96,ghost orchard,0,all in,https://open.spotify.com/track/58tzJQpMWgsPaWm...
97,Prentiss,0,i tried,https://open.spotify.com/track/1lCm0JDZgZ2gyoJ...
98,Red Leather,1,DAKOTA,https://open.spotify.com/track/5WXlPR87hReTmOB...


In [16]:
datos_spotipy_week_2.loc[datos_spotipy_week_2['prediccion']==1].head()


,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,links,prediccion
1,ROSALÍA,LLYLM,0.634,0.555,-5.522,0.12900,0.2730,0.1880,0.555,170.228,174.04,https://open.spotify.com/track/2SiAcexM2p1yX6j...,1
3,Ava Max,Ghost,0.931,0.715,-3.548,0.00573,0.0426,0.0549,0.615,119.971,181.65,https://open.spotify.com/track/7LbhIjYNfCguxjA...,1
5,Popcaan,Aboboyaa (feat. Burna Boy),0.833,0.634,-5.100,0.29700,0.2500,0.3670,0.714,96.881,166.80,https://open.spotify.com/track/2KEMHoK7Tz7rlTU...,1
6,Daniel Caesar,Do You Like Me?,0.407,0.714,-6.806,0.52200,0.0422,0.1280,0.358,76.978,227.66,https://open.spotify.com/track/6q4yusLwjDYEcF6...,1
8,Rauw Alejandro,PANTIES Y BRASIERES,0.888,0.758,-2.851,0.34000,0.0706,0.1440,0.408,107.039,197.38,https://open.spotify.com/track/0fADaWdT3obxk7z...,1


In [17]:
datos_spotipy_week_2[['artist_name','prediccion','track_name','links']].head()

,artist_name,prediccion,track_name,links
0,Sam Smith,0,I’m Not Here To Make Friends (feat. Calvin Har...,https://open.spotify.com/track/3i0FkJYlU4MFfYk...
1,ROSALÍA,1,LLYLM,https://open.spotify.com/track/2SiAcexM2p1yX6j...
2,The Kid LAROI,0,Love Again,https://open.spotify.com/track/4sx6NRwL6Ol3V6m...
3,Ava Max,1,Ghost,https://open.spotify.com/track/7LbhIjYNfCguxjA...
4,Cordae,0,Two Tens (feat. Anderson .Paak),https://open.spotify.com/track/6clDsO8HwhHEgJD...


In [18]:
df_links = datos_spotipy_week_2.loc[datos_spotipy_week_2['prediccion']== 1]



In [19]:
df_links = df_links.reset_index(drop=True)
df_links.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,links,prediccion
0,ROSALÍA,LLYLM,0.634,0.555,-5.522,0.12900,0.2730,0.1880,0.555,170.228,174.04,https://open.spotify.com/track/2SiAcexM2p1yX6j...,1
1,Ava Max,Ghost,0.931,0.715,-3.548,0.00573,0.0426,0.0549,0.615,119.971,181.65,https://open.spotify.com/track/7LbhIjYNfCguxjA...,1
2,Popcaan,Aboboyaa (feat. Burna Boy),0.833,0.634,-5.100,0.29700,0.2500,0.3670,0.714,96.881,166.80,https://open.spotify.com/track/2KEMHoK7Tz7rlTU...,1
3,Daniel Caesar,Do You Like Me?,0.407,0.714,-6.806,0.52200,0.0422,0.1280,0.358,76.978,227.66,https://open.spotify.com/track/6q4yusLwjDYEcF6...,1
4,Rauw Alejandro,PANTIES Y BRASIERES,0.888,0.758,-2.851,0.34000,0.0706,0.1440,0.408,107.039,197.38,https://open.spotify.com/track/0fADaWdT3obxk7z...,1


# Saco la prediccion y los links para el dataframe de España

In [20]:
datos_spotipy_week_2_sp = pd.read_csv('../Data/CSV_full/data_hackshow_spain.csv')

In [21]:
datos_spotipy_week_2normsp = datos_spotipy_week_2_sp.copy()

In [22]:
datos_spotipy_week_2normsp['artist_name']=LabelEncoder().fit_transform(datos_spotipy_week_2normsp['artist_name'])


In [23]:
datos_spotipy_week_2normsp[['energy','danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration','artist_name']] = scaler.fit_transform(datos_spotipy_week_2normsp[['energy',\
                    'danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration','artist_name']])

In [24]:
datos_spotipy_week_2normsp.drop(['track_name','links'],axis=1,inplace=True)

In [25]:
pickled_model_sp = pickle.load(open('../Data/CSV_primeros/GBC.pkl', 'rb'))


In [26]:
y_pred_sp = pickled_model_sp.predict(datos_spotipy_week_2normsp)

In [27]:
pred_sp= pd.Series(y_pred_sp) 

In [28]:
datos_spotipy_week_2_sp['prediccion'] = pred_sp.round(decimals = 0)

In [29]:
datos_spotipy_week_2_sp.prediccion.value_counts()

0    42
1    18
Name: prediccion, dtype: int64

In [30]:
datos_spotipy_week_2_sp[['artist_name','prediccion','track_name','links']].head()

,artist_name,prediccion,track_name,links
0,ROSALÍA,0,LLYLM,https://open.spotify.com/track/2SiAcexM2p1yX6j...
1,Rauw Alejandro,1,PANTIES Y BRASIERES,https://open.spotify.com/track/0fADaWdT3obxk7z...
2,Marc Seguí,0,Plaza en el cielo,https://open.spotify.com/track/330mS1U6KJ9Gie5...
3,Daviles de Novelda,0,Te Miro A La Cara,https://open.spotify.com/track/7hlPoVZ1uz7FiQa...
4,Morad,0,Cuzi Cuza,https://open.spotify.com/track/3ZEcE3DPL1DdDFq...


In [31]:
df_links_sp = datos_spotipy_week_2_sp.loc[datos_spotipy_week_2_sp['prediccion']== 1]


In [32]:
df_links_sp = df_links_sp.reset_index(drop=True)

In [33]:
df_links_sp.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,links,prediccion
0,Rauw Alejandro,PANTIES Y BRASIERES,0.888,0.758,-2.851,0.340,0.0706,0.1440,0.408,107.039,197.38,https://open.spotify.com/track/0fADaWdT3obxk7z...,1
1,Sam Smith,I’m Not Here To Make Friends (feat. Calvin Har...,0.895,0.698,-4.439,0.172,0.0628,0.4090,0.838,114.914,229.60,https://open.spotify.com/track/3i0FkJYlU4MFfYk...,1
2,Christian Nodal,Por el Resto de Tu Vida,0.662,0.781,-3.825,0.355,0.0336,0.1610,0.686,91.999,197.48,https://open.spotify.com/track/40vzZFWG1h60TdZ...,1
3,Las Villa,Bendita,0.765,0.675,-3.935,0.276,0.1180,0.0727,0.596,103.902,199.14,https://open.spotify.com/track/3vDP0LdVBqGhKvz...,1
4,Andrés Suárez,POR NO DECIR TU NOMBRE,0.706,0.766,-6.080,0.156,0.2030,0.1570,0.762,97.990,213.25,https://open.spotify.com/track/3Kc4xk6kNmSgM6A...,1


In [34]:
links_final = pd.concat([df_links,df_links_sp]).reset_index(drop=True)



In [35]:
links_final.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,links,prediccion
0,ROSALÍA,LLYLM,0.634,0.555,-5.522,0.12900,0.2730,0.1880,0.555,170.228,174.04,https://open.spotify.com/track/2SiAcexM2p1yX6j...,1
1,Ava Max,Ghost,0.931,0.715,-3.548,0.00573,0.0426,0.0549,0.615,119.971,181.65,https://open.spotify.com/track/7LbhIjYNfCguxjA...,1
2,Popcaan,Aboboyaa (feat. Burna Boy),0.833,0.634,-5.100,0.29700,0.2500,0.3670,0.714,96.881,166.80,https://open.spotify.com/track/2KEMHoK7Tz7rlTU...,1
3,Daniel Caesar,Do You Like Me?,0.407,0.714,-6.806,0.52200,0.0422,0.1280,0.358,76.978,227.66,https://open.spotify.com/track/6q4yusLwjDYEcF6...,1
4,Rauw Alejandro,PANTIES Y BRASIERES,0.888,0.758,-2.851,0.34000,0.0706,0.1440,0.408,107.039,197.38,https://open.spotify.com/track/0fADaWdT3obxk7z...,1


In [36]:
links_final.shape

(62, 13)

In [37]:
links_final.drop_duplicates().shape==links_final.shape    

False

In [38]:
links_final=links_final.drop_duplicates()


In [39]:
links_final.drop_duplicates().shape==links_final.shape    

True

In [40]:
links_final.shape

(60, 13)

In [41]:
links = links_final.links

In [42]:
links = links.tolist()

In [43]:
links

['https://open.spotify.com/track/2SiAcexM2p1yX6joESbehd',
 'https://open.spotify.com/track/7LbhIjYNfCguxjAw43Mkff',
 'https://open.spotify.com/track/2KEMHoK7Tz7rlTUalQXMUk',
 'https://open.spotify.com/track/6q4yusLwjDYEcF6jXHdt7J',
 'https://open.spotify.com/track/0fADaWdT3obxk7zvHS18VY',
 'https://open.spotify.com/track/5t6VBNWXxcwlrxCjU7ytGg',
 'https://open.spotify.com/track/4FWbsd91QSvgr1dSWwW51e',
 'https://open.spotify.com/track/2Lq7rOLcBBTvBa86k1YTW0',
 'https://open.spotify.com/track/06dQ2rcBkq7z3AEZ4jhvc2',
 'https://open.spotify.com/track/00I5NjPGm836J2MQmYxhs9',
 'https://open.spotify.com/track/5XYk13uT6mzwe6P8lSrPdh',
 'https://open.spotify.com/track/6pn5dBDetgg4rojbzIxH7v',
 'https://open.spotify.com/track/1JXoYzZXVW7H829mHfiWRm',
 'https://open.spotify.com/track/6DE1vkPkMn2I77360IBb8t',
 'https://open.spotify.com/track/6ZBr8E9XczoqgMRXyXUe6b',
 'https://open.spotify.com/track/2LWJ0zNei0z2WjHHwSNUkx',
 'https://open.spotify.com/track/35XF6veSvhoPdWy89zaeaj',
 'https://open

In [50]:
token = 'BQCJuOGd2gOxwez3bEG1vqJN3JHH4_LaCLqFnvr0Dc-ZK5vOmW0VXZbQRzRA7HY7WAnnyFEJYfyri9s3cXj3PvckGPL19Oc3CjmTYhnh2u6CI-1t2tUR4NJksy8_xhmKqznVL55aZUm3P0V4-1sgqsN9Ol3H7_aEV9FAEyiyTEIOu3ZVYMQyGeVpansgDecupYSMqWOwRHghXA'

In [51]:
sp = spotipy.Spotify(auth=token)

In [52]:
sp.user_playlist_create(user='javi1025',name='Hackshow',description='playlist para el Hackshow')

{'collaborative': False,
 'description': 'playlist para el Hackshow',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/3SKaeqp9TUYolgY4s2k1bo'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/3SKaeqp9TUYolgY4s2k1bo',
 'id': '3SKaeqp9TUYolgY4s2k1bo',
 'images': [],
 'name': 'Hackshow',
 'owner': {'display_name': 'javi1025',
  'external_urls': {'spotify': 'https://open.spotify.com/user/javi1025'},
  'href': 'https://api.spotify.com/v1/users/javi1025',
  'id': 'javi1025',
  'type': 'user',
  'uri': 'spotify:user:javi1025'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSw4YTZmYzMwNDI5OTU3ZjljZDFhZmE1NWJhZWJiYzQ5ZGRlZGY1MDUz',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/3SKaeqp9TUYolgY4s2k1bo/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:3SKaeqp9TUYolgY4s2k1bo'}

In [53]:
sp.user_playlist_add_tracks(user='javi1025', 
                            playlist_id='3SKaeqp9TUYolgY4s2k1bo',
                            tracks=links, 
                            position=None)

{'snapshot_id': 'MywxMTgyNDA5NDJhZWQzZTI0MTE0NDhhMDBjZmE4ODM4ZDg0NjFmYWM0'}

# Crear dataframe de exitos españa y globales juntos para ver sus caracteristicas

In [55]:
a = pd.read_csv('../Data/CSV_primeros/all_songs.csv')

In [56]:
a.top_hit.value_counts()

1    3785
Name: top_hit, dtype: int64

In [57]:
b = pd.read_csv('../Data/CSV_primeros/spain_hackshow.csv')

In [58]:
b.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,top_hit
0,Coldplay,Yellow,0.661,0.429,-7.227,0.00239,0.0281,0.2340,0.285,173.372,266.77,1
1,Amaral,Cómo hablar,0.775,0.606,-4.752,0.03560,0.0291,0.1110,0.626,117.357,241.27,1
2,Eminem,The Real Slim Shady,0.661,0.949,-4.244,0.03020,0.0572,0.0454,0.760,104.504,284.20,1
3,Britney Spears,Oops!...I Did It Again,0.834,0.751,-5.444,0.30000,0.0437,0.3550,0.894,95.053,211.16,1
4,La Oreja de Van Gogh,La Playa,0.552,0.589,-5.707,0.17500,0.0268,0.1060,0.311,88.535,247.69,1


In [59]:
all_hits = pd.concat([a,b]).reset_index(drop=True)


In [60]:
all_hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5835 entries, 0 to 5834
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   artist_name   5835 non-null   object 
 1   track_name    5835 non-null   object 
 2   energy        5835 non-null   object 
 3   danceability  5835 non-null   object 
 4   loudness      5835 non-null   object 
 5   acousticness  5835 non-null   object 
 6   speechiness   5835 non-null   object 
 7   liveness      5835 non-null   object 
 8   valence       5835 non-null   object 
 9   tempo         5835 non-null   object 
 10  duration      5824 non-null   float64
 11  top_hit       5835 non-null   int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 547.2+ KB


In [61]:
all_hits['energy'] = pd.to_numeric(all_hits['energy'], errors='coerce')


In [62]:
all_hits['danceability'] = pd.to_numeric(all_hits['danceability'], errors='coerce')

In [63]:
all_hits['loudness'] = pd.to_numeric(all_hits['loudness'], errors='coerce')


In [64]:
all_hits['acousticness'] = pd.to_numeric(all_hits['acousticness'], errors='coerce')


In [65]:
all_hits['speechiness'] = pd.to_numeric(all_hits['speechiness'], errors='coerce')


In [66]:
all_hits['liveness'] = pd.to_numeric(all_hits['liveness'], errors='coerce')


In [67]:
all_hits['valence'] = pd.to_numeric(all_hits['valence'], errors='coerce')


In [68]:
all_hits['tempo'] = pd.to_numeric(all_hits['tempo'], errors='coerce')


In [69]:
all_hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5835 entries, 0 to 5834
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   artist_name   5835 non-null   object 
 1   track_name    5835 non-null   object 
 2   energy        5824 non-null   float64
 3   danceability  5824 non-null   float64
 4   loudness      5824 non-null   float64
 5   acousticness  5824 non-null   float64
 6   speechiness   5824 non-null   float64
 7   liveness      5824 non-null   float64
 8   valence       5824 non-null   float64
 9   tempo         5824 non-null   float64
 10  duration      5824 non-null   float64
 11  top_hit       5835 non-null   int64  
dtypes: float64(9), int64(1), object(2)
memory usage: 547.2+ KB


In [70]:
all_hits.drop_duplicates().shape==all_hits.shape    

False

In [74]:
all_hits.drop_duplicates().shape==all_hits.shape    

True

In [75]:
all_hits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5219 entries, 0 to 5834
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   artist_name   5219 non-null   object 
 1   track_name    5219 non-null   object 
 2   energy        5208 non-null   float64
 3   danceability  5208 non-null   float64
 4   loudness      5208 non-null   float64
 5   acousticness  5208 non-null   float64
 6   speechiness   5208 non-null   float64
 7   liveness      5208 non-null   float64
 8   valence       5208 non-null   float64
 9   tempo         5208 non-null   float64
 10  duration      5208 non-null   float64
 11  top_hit       5219 non-null   int64  
dtypes: float64(9), int64(1), object(2)
memory usage: 530.1+ KB


In [73]:
all_hits=all_hits.drop_duplicates()

In [76]:
all_hits.dropna(inplace=True)

In [ ]:
all_hits.describe().T

In [77]:
all_hits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5208 entries, 0 to 5834
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   artist_name   5208 non-null   object 
 1   track_name    5208 non-null   object 
 2   energy        5208 non-null   float64
 3   danceability  5208 non-null   float64
 4   loudness      5208 non-null   float64
 5   acousticness  5208 non-null   float64
 6   speechiness   5208 non-null   float64
 7   liveness      5208 non-null   float64
 8   valence       5208 non-null   float64
 9   tempo         5208 non-null   float64
 10  duration      5208 non-null   float64
 11  top_hit       5208 non-null   int64  
dtypes: float64(9), int64(1), object(2)
memory usage: 528.9+ KB


In [78]:
unpopular = pd.read_csv('../Data/Spoti/unpopular_songs.csv')

unpopular.shape

(10877, 17)

In [79]:
unpopular.drop(['explicit','mode','popularity','key','track_id','instrumentalness'],axis=1,inplace=True)

In [80]:
unpopular=unpopular.iloc[:, [10,9,1,0,2,4,3,5,6,7,8]]


In [81]:
unpopular.rename(columns = {'track_artist':'artist_name', 'track_name':'track_name','duration_ms':'duration'}, inplace = True)

In [82]:
unpopular['top_hit']=[0 for i in range(len(unpopular))]

In [83]:
unpopular['duration'] = (unpopular['duration']/1000).round(2)

unpopular.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,top_hit
0,James Reeder,No Regrets,0.770,0.530,-6.633,0.284,0.0389,0.744,0.623,120.144,225.70,0
1,James Reeder,Wild Life,0.730,0.565,-6.063,0.365,0.0730,0.237,0.511,130.026,158.09,0
2,James Reeder,Fangs,0.546,0.427,-8.727,0.539,0.0849,0.368,0.435,78.345,167.26,0
3,James Reeder,Afterburner,0.531,0.421,-5.516,0.706,0.0262,0.110,0.383,85.080,236.83,0
4,James Reeder,Hellfire Rising,0.804,0.537,-7.378,0.379,0.1570,0.323,0.543,139.950,239.40,0


In [84]:
unpopular = unpopular.sample(n=5208)

In [88]:
unpopular = unpopular.reset_index(drop=True)

In [89]:
unpopular

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,top_hit
0,Futurecop!,Starworshipper (MPM remix),0.8550,0.546,-4.781,0.18000,0.0330,0.1150,0.312,126.000,301.00,0
1,Bollobas,Reckless Pleasure,0.0985,0.346,-18.920,0.54700,0.0332,0.2410,0.249,187.984,78.11,0
2,Ron Reeser,Everything,0.7240,0.728,-8.287,0.00449,0.0608,0.2970,0.724,130.035,236.32,0
3,Storyhill,She Holds My Heart out in the Wind,0.4710,0.404,-9.148,0.61600,0.0257,0.2020,0.561,91.982,196.48,0
4,Ian Brown,The Feeding Of The 5000,0.5260,0.485,-9.484,0.30100,0.0466,0.1180,0.112,81.983,238.36,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5203,Kandi,Don't Think I'm Not - Hip Hop Remix,0.6610,0.698,-8.599,0.05410,0.0774,0.0627,0.600,97.855,274.23,0
5204,Alex G,Lighthouse,0.3900,0.602,-8.967,0.81900,0.0774,0.1020,0.316,99.343,557.00,0
5205,El Gran Combo De Puerto Rico,Mi Gorda Bonita,0.7710,0.633,-3.775,0.40900,0.0601,0.0382,0.962,99.988,320.51,0
5206,Slumm Prince,Mobbing,0.5370,0.860,-7.570,0.32400,0.2690,0.1260,0.446,123.952,233.33,0


In [90]:
canciones_hackshow = pd.concat([all_hits,unpopular]).reset_index(drop=True)


In [91]:
canciones_hackshow

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,top_hit
0,Britney Spears,Oops!...I Did It Again,0.834,0.751,-5.444,0.3000,0.0437,0.3550,0.894,95.053,211.16,1
1,blink-182,All The Small Things,0.897,0.434,-4.918,0.0103,0.0488,0.6120,0.684,148.726,167.07,1
2,Faith Hill,Breathe,0.496,0.529,-9.007,0.1730,0.0290,0.2510,0.278,136.859,250.55,1
3,Bon Jovi,It's My Life,0.913,0.551,-4.063,0.0263,0.0466,0.3470,0.544,119.992,224.49,1
4,*NSYNC,Bye Bye Bye,0.928,0.614,-4.806,0.0408,0.0516,0.0845,0.879,172.656,200.56,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10411,Kandi,Don't Think I'm Not - Hip Hop Remix,0.661,0.698,-8.599,0.0541,0.0774,0.0627,0.600,97.855,274.23,0
10412,Alex G,Lighthouse,0.390,0.602,-8.967,0.8190,0.0774,0.1020,0.316,99.343,557.00,0
10413,El Gran Combo De Puerto Rico,Mi Gorda Bonita,0.771,0.633,-3.775,0.4090,0.0601,0.0382,0.962,99.988,320.51,0
10414,Slumm Prince,Mobbing,0.537,0.860,-7.570,0.3240,0.2690,0.1260,0.446,123.952,233.33,0


In [92]:
canciones_hackshow.to_csv('../Hackshow/Dataframe.csv', index=False)

In [93]:
links.to_csv('../Hackshow/links_playlist.csv', index=False)
links

AttributeError: 'list' object has no attribute 'to_csv'

In [95]:
unpopular.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,top_hit
0,Futurecop!,Starworshipper (MPM remix),0.8550,0.546,-4.781,0.18000,0.0330,0.115,0.312,126.000,301.00,0
1,Bollobas,Reckless Pleasure,0.0985,0.346,-18.920,0.54700,0.0332,0.241,0.249,187.984,78.11,0
2,Ron Reeser,Everything,0.7240,0.728,-8.287,0.00449,0.0608,0.297,0.724,130.035,236.32,0
3,Storyhill,She Holds My Heart out in the Wind,0.4710,0.404,-9.148,0.61600,0.0257,0.202,0.561,91.982,196.48,0
4,Ian Brown,The Feeding Of The 5000,0.5260,0.485,-9.484,0.30100,0.0466,0.118,0.112,81.983,238.36,0
